In [15]:
import sys
{sys.prefix}

{'/opt/homebrew/anaconda3/envs/pipeline'}

In [13]:
!conda install -c conda-forge -y --prefix {sys.prefix} beautifulsoup4
!conda install -c conda-forge -y --prefix {sys.prefix} lxml
!conda install -c conda-forge -y --prefix {sys.prefix} matplotlib
!conda install -c conda-forge -y --prefix {sys.prefix} html5lib


EnvironmentLocationNotFound: Not a conda environment: /Users/mdgome/workspace/study/StudyDataEngineering/StudyDataEngineering/CarZoom/API/{sys.prefix}


EnvironmentLocationNotFound: Not a conda environment: /Users/mdgome/workspace/study/StudyDataEngineering/StudyDataEngineering/CarZoom/API/{sys.prefix}


EnvironmentLocationNotFound: Not a conda environment: /Users/mdgome/workspace/study/StudyDataEngineering/StudyDataEngineering/CarZoom/API/{sys.prefix}


EnvironmentLocationNotFound: Not a conda environment: /Users/mdgome/workspace/study/StudyDataEngineering/StudyDataEngineering/CarZoom/API/{sys.prefix}



In [16]:
import pandas as pd
from bs4 import BeautifulSoup
from lxml import html
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote, unquote_plus
import matplotlib 
import requests

In [17]:
matplotlib.rcParams['axes.unicode_minus'] = False 
matplotlib.rcParams['font.family'] = "NanumGothic"

In [18]:
# 1. URL 파라미터 분리하기.
# Service URL
xmlUrl = 'http://openapi.tago.go.kr/openapi/service/CarSharingInfoService/getCarZoneListByName'
My_API_Key = 'dyJVzBeXHhkzJZHXpnaRzc6GvovKIXadt2bOtynDyUBuJjWk0VBG8NBk8vyPIfWEZ0yb1CJ6K+ep3EyZj89ZvA=='

queryParams = '?' + urlencode(    # get 방식으로 쿼리를 분리하기 위해 '?'를 넣은 것이다. 메타코드 아님.
    {
        quote_plus('ServiceKey') : My_API_Key,    # 필수 항목 1 : 서비스키 (본인의 서비스키)
        #quote_plus('pageNo') : 1,          # 필수 항목 2 : 페이지 번호
        #quote_plus('numOfRows') : 50,         # 픽수 항목 3 : 한 페이지 결과 수
        quote_plus('zoneName') : "서울역"         # 픽수 항목 3 : 한 페이지 결과 수
     }
)

In [19]:
getUrl= unquote_plus(xmlUrl+queryParams)
getUrl #디버깅용

'http://openapi.tago.go.kr/openapi/service/CarSharingInfoService/getCarZoneListByName?ServiceKey=dyJVzBeXHhkzJZHXpnaRzc6GvovKIXadt2bOtynDyUBuJjWk0VBG8NBk8vyPIfWEZ0yb1CJ6K+ep3EyZj89ZvA==&zoneName=서울역'

In [20]:
response = requests.get(xmlUrl + queryParams).text.encode('utf-8')
xmlobj = BeautifulSoup(response, 'lxml-xml')
xmlobj    # 디버깅용.

<?xml version="1.0" encoding="utf-8"?>
<response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><address>서울 중구 남대문로5가 581</address><latitude>37.554298</latitude><longitude>126.973892</longitude><type>1</type><zoneId>S0000808</zoneId><zoneName>서울역 10번출구(서울시티타워)(운영종료)</zoneName></item><item><address>서울 용산구 동자동 43-205</address><latitude>37.555862</latitude><longitude>126.970505</longitude><type>2</type><zoneId>10519</zoneId><zoneName>서울역사</zoneName></item><item><address>서울 중구 남대문로5가 537</address><latitude>37.556099</latitude><longitude>126.974869</longitude><type>2</type><zoneId>10544</zoneId><zoneName>서울역 8번출구</zoneName></item><item><address>서울 용산구 동자동 45</address><latitude>37.551563</latitude><longitude>126.973236</longitude><type>2</type><zoneId>10981</zoneId><zoneName>서울역 12번출구(KDB생명타워)</zoneName></item><item><address>서울 용산구 서계동 47-3</address><latitude>37.55228</latitude><longitude>126.968849</longitude><type>2</type><zoneId>1754</

In [21]:
rows = xmlobj.findAll('item')
# rows    # 디버깅용.
# rows[0]    # 디버깅용.

In [22]:
columns = rows[0].find_all()
# columns    # 디버깅용.
# columns[0].name    # 디버깅용.
# columns[0].text    # 디버깅용.

In [23]:
# 모든 행과 열의 값을 모아 매트릭스로 만들어보자.
rowList = []
nameList = []
columnList = []

rowsLen = len(rows)
for i in range(0, rowsLen):
    columns = rows[i].find_all()
    
    columnsLen = len(columns)
    for j in range(0, columnsLen):
        # 첫 번째 행 데이터 값 수집 시에만 컬럼 값을 저장한다. (어차피 rows[0], rows[1], ... 모두 컬럼헤더는 동일한 값을 가지기 때문에 매번 반복할 필요가 없다.)
        if i == 0:
            nameList.append(columns[j].name)
        # 컬럼값은 모든 행의 값을 저장해야한다.    
        eachColumn = columns[j].text
        columnList.append(eachColumn)
    rowList.append(columnList)
    columnList = []    # 다음 row의 값을 넣기 위해 비워준다. (매우 중요!!)
    
result = pd.DataFrame(rowList, columns=nameList)
result.head()

,address,latitude,longitude,type,zoneId,zoneName
0,서울 중구 남대문로5가 581,37.554298,126.973892,1,S0000808,서울역 10번출구(서울시티타워)(운영종료)
1,서울 용산구 동자동 43-205,37.555862,126.970505,2,10519,서울역사
2,서울 중구 남대문로5가 537,37.556099,126.974869,2,10544,서울역 8번출구
3,서울 용산구 동자동 45,37.551563,126.973236,2,10981,서울역 12번출구(KDB생명타워)
4,서울 용산구 서계동 47-3,37.55228,126.968849,2,1754,롯데렌터카 서울역영업소 2층


In [30]:
def company():
    if x == 1:
        return "쏘카"
    elif x == 2:
        return "그린카"
    else:
        return None

In [31]:
result["company"] = result["type"].apply(lambda x : company(x))

TypeError: company() takes 0 positional arguments but 1 was given

In [ ]:
result